In [1]:
%run cm_queryset_google_topics.py

Baseline queryset pushed forward from the fatalities002, new updated version.
 Topics has been slightly changed to reflect the new time lag. Note in the future we should be able to tlag by 1 only as for example, by September 7th we should have all data for October. So technically ahead of the ucdp update schedule. For right now, though, we have data through July so given that we are running the model in October I lagged the value by 3 instead of 1. I converted the 1 year lag to 15 months, and then replaced the running average by 3 month lag with stock over 12. Note that the running average is not the exact version as is calculated by Rauh and Muller team but a stand in. Additional variables will be added.
 Google trends has similarly been lagged to reflect the last available data is August. There might be something wrong with my querysets where the NAs are getting autofilled with 0s when querying. So, I am setting the google_index value by tlag 2. 
 Internet usage is available at count

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views

In [ ]:
!viewser tables show internet_usage_our_world_data_cy

In [2]:
get_cm_querysets()

 .    fat_dev_mc_media_baseline; A dataset with 6 columns, with data between t 1 and 852. (213 units)
 .    fat_dev_mc_media_topics; A dataset with 60 columns, with data between t 1 and 852. (213 units)
 .    fat_dev_mc_media_google_internet; A dataset with 13 columns, with data between t 1 and 852. (213 units)
